## scraping

In [47]:
from bs4 import BeautifulSoup

html_doc = """

<table class="table-striped">
<thead>
			<tr>
				<td width="1"></td>
								<th>Owner</th>
				<th>Name</th>
				<th>Location</th>
				<th>&nbsp;</th>
			</tr>
		</thead>
<tbody>
						<tr>
			<td class="status_green"></td>
						<td>
					Aanevoima Oy			</td>
			<td>
				
					Aanekoski power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3641/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Aarhus University			</td>
			<td>
				
					Center for Biorefining Technologies				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3726/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Advanced Biofuels Solutions Ltd			</td>
			<td>
				
					Swindon Advanced Biofuels Plant				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3717/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Advanced Biofuels Solutions Ltd			</td>
			<td>
				
					ABSL bio-SNG demonstrator				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3886/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Advanced Biofuels Solutions Ltd			</td>
			<td>
				
					Swindon Advanced Biofuels Plant				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3717/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Aemetis			</td>
			<td>
				
					Aemetis Carbon Zero 1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3774/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>

		
			</tbody>
</table>
"""



In [54]:
html_doc="""
<table class="table-striped">
		<thead>
			<tr>
				<td width="1"></td>
								<th>Owner</th>
				<th>Name</th>
				<th>Location</th>
				<th>&nbsp;</th>
			</tr>
		</thead>
		<tbody>
						<tr>
			<td class="status_green"></td>
						<td>
					Aanevoima Oy			</td>
			<td>
				
					Aanekoski power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3641/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Aarhus University			</td>
			<td>
				
					Center for Biorefining Technologies				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3726/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Advanced Biofuels Solutions Ltd			</td>
			<td>
				
					Swindon Advanced Biofuels Plant				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3717/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Advanced Biofuels Solutions Ltd			</td>
			<td>
				
					ABSL bio-SNG demonstrator				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3886/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Advanced Biofuels Solutions Ltd			</td>
			<td>
				
					Swindon Advanced Biofuels Plant				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3717/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Aemetis			</td>
			<td>
				
					Aemetis Carbon Zero 1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3774/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Aemetis/Lanzatech			</td>
			<td>
				
					Project Aemetis Riverbank				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3844/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Aerni Pratteln			</td>
			<td>
				
					CHP Pratteln				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3061/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					AEW Energie AG			</td>
			<td>
				
					Pelletvergasser AEW Rheinfelden				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3701/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					AEW UK			</td>
			<td>
				
					Hoddesdon Advanced Thermal Treatment				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3970/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					AgBioEn			</td>
			<td>
				
					Australian renewable energy facility				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3969/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Agnion Technologies GmbH			</td>
			<td>
				
					CHP Agnion Biomasse Heizkraftwerk Pfaffenhofen				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3065/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ALTACA ENERGY  			</td>
			<td>
				
					ALTACA ENERGY  				
			</td>
			<td>
					Turkey                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3904/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Alternative Energy Solutions Ltd.			</td>
			<td>
				
					Alternative Energy Solutions Ltd.				
			</td>
			<td>
					New Zealand                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3405/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Amyris, Inc.			</td>
			<td>
				
					Amyris Tate &amp; Lyle				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3256/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Anhui BBCA Biochemical			</td>
			<td>
				
					BBCA				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3353/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Anhui Guozhen Group and Chemtex Chemical Engineering			</td>
			<td>
				
					Fuyang project				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3850/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					AquaGreen ApS			</td>
			<td>
				
					AquaGreen/Farevejle wastewater facility 				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3938/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					AquaGreen ApS			</td>
			<td>
				
					AquaGreen PCE				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3937/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Arbaflame			</td>
			<td>
				
					Arbaflame Grasmo plant				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3794/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Arbios Biotech (Joint Venture of Licella and Canfor)			</td>
			<td>
				
					CS-1 Facility				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3992/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Arbios Biotech (JV Canfor Licella)			</td>
			<td>
				
					Low Carbon Biofuel Plant				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3809/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Arbor Renewable Gas LLC			</td>
			<td>
				
					Spindletop Plant				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4013/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					ARBRE Energy Limited (AEL)			</td>
			<td>
				
					IGCC ARBRE Energy Eggborough				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3087/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ArcelorMittal			</td>
			<td>
				
					Ghent Steelanol				
			</td>
			<td>
					Belgium                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3767/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ARD			</td>
			<td>
				
					Futurol pilot				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3239/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ASM BRESCIA S.P.A.			</td>
			<td>
				
					Termoutilizzatore				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3499/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Assam Bio Refinery (ABRPL)			</td>
			<td>
				
					2G ethanol Commercial plant				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3945/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Associated Electric Cooperative, Inc.			</td>
			<td>
				
					Thomas Hill Energy Center #2				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3579/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Audi AG			</td>
			<td>
				
					Audi e-gas				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3954/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					AustroCel Hallein			</td>
			<td>
				
					biorefinery				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3241/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Autogasnord			</td>
			<td>
				
					-				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3131/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Azienda agricola Camardo			</td>
			<td>
				
					-				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3130/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Azienda Agricola Isca di Calvello			</td>
			<td>
				
					Urbas Calvello				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3159/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Azienda Agricola San Vittore			</td>
			<td>
				
					-				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3134/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Azienda Agri S.A.B.I.			</td>
			<td>
				
					Azienda Agri				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3449/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Azienda Agri S.A.B.I.			</td>
			<td>
				
					Azienda Agri				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3445/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Azienda Agri S.A.B.I.			</td>
			<td>
				
					Azienda Agri				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3446/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Azienda Agri S.A.B.I.			</td>
			<td>
				
					Azienda Agri				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3447/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Azienda Agri S.A.B.I.			</td>
			<td>
				
					Azienda Agri				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3448/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Azienda Tenca dei Fratelli Zanotti/AB energy			</td>
			<td>
				
					Orzinuovi				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3127/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Azienda Tessile Parmense			</td>
			<td>
				
					GAS 1000				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3136/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Baas Energie BV			</td>
			<td>
				
					Kombi Power System Baas				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3857/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Babcock&amp;Wilcox Volund			</td>
			<td>
				
					CHP Updraft gasifier Yamagata				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3073/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Babcock&amp;Wilcox Volund			</td>
			<td>
				
					CHP B&amp;W Harboore				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3072/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Babcock&amp;Wilcox Volund			</td>
			<td>
				
					CHP Updraft gasifier Daio				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3074/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Bayernwerk			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3481/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Bayernwerke			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3492/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ba Yu Paper			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Taiwan                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3452/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Beijing Shougang LanzaTech New Energy Technology Co., Ltd			</td>
			<td>
				
					Shougang Commercial				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3773/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BEST - Bioenergy and Sustainable Technologies GmbH			</td>
			<td>
				
					Waste2Value				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4024/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Beta Renewables			</td>
			<td>
				
					Fujiang Bioproject				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3352/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Beta Renewables			</td>
			<td>
				
					Canergy				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3341/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BEWAG			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3489/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Bharat Petroleum			</td>
			<td>
				
					2G ethanol Commercial plant				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3947/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Bioenergie Schnellingen			</td>
			<td>
				
					Bioenergie Schnellingen				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3740/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					bioenergy 2020+			</td>
			<td>
				
					One Barrel per Day Pilot Plant				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3790/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					BIOGAS ENERGY 			</td>
			<td>
				
					BIOGAS ENERGY PROJECT				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3905/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					BioGasol			</td>
			<td>
				
					BornBioFuel 2				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3222/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Biojet AS			</td>
			<td>
				
					Liquid Biofuels Facility Follum				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4006/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Biomasse Energie GmbH			</td>
			<td>
				
					FICFB Villach				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3081/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Biomaterial in Tokyo Co., Ltd.			</td>
			<td>
				
					Bio-Jet production development (with NEDO) 				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3950/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BioMCN (OCI NV)			</td>
			<td>
				
					BioMCN 				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3181/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Bio SNG Guessing			</td>
			<td>
				
					Synthesis Demo Guessing				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3035/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					BioSynergi Proces ApS			</td>
			<td>
				
					BioSynergi CHP demonstration plant				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3106/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BioTfueL-consortium			</td>
			<td>
				
					BioTfueL pilot				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3302/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Biowaerme Eberndorf			</td>
			<td>
				
					CHP Urbas Eberndorf				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3100/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Biowaerme Mallnitz GmbH			</td>
			<td>
				
					Urbas Mallnitz				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3155/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Bio&amp;Watt			</td>
			<td>
				
					-				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3132/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Biozin			</td>
			<td>
				
					Biozin biocrude				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3839/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Birmingham Bio-power			</td>
			<td>
				
					Birmingham Bio-power				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3887/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BKB			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3478/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Black River Partners			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3550/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Blue Energy Syngas GmbH			</td>
			<td>
				
					Holzheizkraftwerk Senden				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3067/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Bord na Mona			</td>
			<td>
				
					Edenderry Power Plant				
			</td>
			<td>
					Ireland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3705/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Borregaard AS			</td>
			<td>
				
					BALI Biorefinery Demo				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3248/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Borregaard Industries AS			</td>
			<td>
				
					ChemCell Ethanol				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3243/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BP			</td>
			<td>
				
					Co-processing Castellon				
			</td>
			<td>
					Spain                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3995/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BP			</td>
			<td>
				
					Cherry Point refinery				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3985/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BP			</td>
			<td>
				
					Co-processing Castellon				
			</td>
			<td>
					Spain                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3995/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					BP Biofuels			</td>
			<td>
				
					BP Biofuels				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3348/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Brista Kraft AB			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3509/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					British Energy			</td>
			<td>
				
					Eggborough				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3530/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BTG			</td>
			<td>
				
									
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3407/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BTG BioLiquids (BTG-BTL)			</td>
			<td>
				
					GREEN FUEL NORDIC OY				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3893/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BTG-BtL			</td>
			<td>
				
					Empyro Enschede				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3818/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					BTG-BTL			</td>
			<td>
				
					EMPYRO				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3408/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Buergerenergie St. Peter			</td>
			<td>
				
					Buergerenergie St. Peter				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3741/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Burkhardt Cham			</td>
			<td>
				
					Burkhardt Cham				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3742/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Burkhardt Neumarkt			</td>
			<td>
				
					Burkhardt Neumarkt				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3743/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Burkhardt Nuernberger Land			</td>
			<td>
				
					Burkhardt Nuernberger Land				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3744/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Caledonian Paper plc			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3524/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Calumet			</td>
			<td>
				
					Revamp Great Falls Refinery				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4010/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Cane Technology Center (CTC)			</td>
			<td>
				
					CTC				
			</td>
			<td>
					Brazil                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3285/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Canfor (CPPI) Pulp Mills and Licella Pty Ltd			</td>
			<td>
				
					CANFOR-LICELLA JOINT VENTURE				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3890/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					CanmetENERGY			</td>
			<td>
				
					CanmetENERGY				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3409/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					CanmetENERGY			</td>
			<td>
				
					CanmetENERGY PDU				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3410/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Cargill and Love's Family of Companies (Joint venture Heartwell Renewables)			</td>
			<td>
				
					Heartwell Renewables facility				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3990/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Centro Cisa			</td>
			<td>
				
					Castel DAAiano				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3126/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Cepsa			</td>
			<td>
				
					Cepsa 2				
			</td>
			<td>
					Spain                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3308/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Cepsa La Rabida research			</td>
			<td>
				
					Cepsa				
			</td>
			<td>
					Spain                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3307/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Chalmers Technical University			</td>
			<td>
				
					Centre for Indirect Gasification of Biomass				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3124/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Chempolis Ltd.			</td>
			<td>
				
					Chempolis Biorefining Plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3250/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					CHOREN Fuel Freiberg GmbH &amp; Co. KG			</td>
			<td>
				
					Synthesis CHOREN beta plant Freiberg 				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3031/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					CHOREN Industries GmbH			</td>
			<td>
				
					Synthesis CHOREN sigma plant Schwedt				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3027/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ciamber			</td>
			<td>
				
					-				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3137/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Clariant			</td>
			<td>
				
					Clariant Romania				
			</td>
			<td>
					Romania                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3798/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Clariant			</td>
			<td>
				
					sunliquid				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3251/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Clariant			</td>
			<td>
				
					Clariant Romania				
			</td>
			<td>
					Romania                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3798/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Clariant			</td>
			<td>
				
					sunliquid				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3251/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					COFCO Zhaodong Co.			</td>
			<td>
				
					COFCO Commercial				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3709/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					COFCO Zhaodong Co.			</td>
			<td>
				
					COFCO Demo				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3359/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Comune Quingentole			</td>
			<td>
				
					-				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3138/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Conoco Philipps			</td>
			<td>
				
					Conoco Philipps				
			</td>
			<td>
					Ireland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3312/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Consortium Toshiba, TOYO, Idemitso, Japan CCS, ANA			</td>
			<td>
				
					Circular carbon SAF demo				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4009/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Consortium TOYO, JERA, Mitsubishi, Itochu			</td>
			<td>
				
					Commercial-scale biomass-to-SAF				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4008/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					CORE Biofuel			</td>
			<td>
				
					Demo plant				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3339/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Corenso coreboard mill			</td>
			<td>
				
					n.a				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3677/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Cortus AB 			</td>
			<td>
				
					WoodRoll Demonstration				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3161/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Cortus Energy AB			</td>
			<td>
				
					Probiostal				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3784/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Covenant Energy			</td>
			<td>
				
					Renewable fuel processing plant 				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3996/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					co-Ver Energy Holding			</td>
			<td>
				
					Lake Maggiore Tecnoparco				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3128/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Cresta			</td>
			<td>
				
					Braya Renewable Fuels				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3997/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					CS Energy			</td>
			<td>
				
					Swanbank B				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3455/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					CSIR-Indian Institute of Petroleum			</td>
			<td>
				
					Biojet fuel pilot 				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3948/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Cutec			</td>
			<td>
				
					Synthesis Cutec Clausthal-Zellerfeld				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3049/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					CVR Energy Inc.			</td>
			<td>
				
					Wynnewood renewable diesel project				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3974/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DBI-Virtuhcon GmbH 			</td>
			<td>
				
					FlexiEntrained (GSP) Pilot Plant				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4023/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Delta Electricity			</td>
			<td>
				
					Mt Piper				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3458/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Delta Electricity			</td>
			<td>
				
					Wallerawang				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3459/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Delta Electricity			</td>
			<td>
				
					Vales Point				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3456/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Diamond Green Diesel			</td>
			<td>
				
					Diamond Green Diesel				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3336/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Dillinger Saar GmbH			</td>
			<td>
				
					Project Selma				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4015/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DINS Sakai Co.,Ltd.			</td>
			<td>
				
					Construction waste timber to ethanol				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3951/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Domsjoe Fabriker			</td>
			<td>
				
					Domsjoe Fabriker				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3319/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Esbjerg Power Station				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3470/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Grenaa Co-Generation Plant				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3471/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Herningvaerket				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3599/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Avedoere #1				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3771/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Studstrupvaerket #3				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3595/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Studstrupvaerket #4				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3469/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Skaerbaekvaerket #3				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3769/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Ensted #3				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3601/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Avedore #2				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3597/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Avedore #2				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3598/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Asnaesvaerket - ASV6				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3770/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					DONG Energy			</td>
			<td>
				
					Studstrupvaerket #1				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3468/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Drax Power			</td>
			<td>
				
					UK				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3529/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DTU Chemical engineering			</td>
			<td>
				
					Viking Gasifier				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3934/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					DTU Chemical engineering			</td>
			<td>
				
					LT-CFB (Low Temperature Circulating Fluidised Bed)				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3935/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Duchi Fratelli Societa Agricola/Agroenergia			</td>
			<td>
				
					-				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3133/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Duke Power Company			</td>
			<td>
				
					Lee (W.S) Steam Station #3				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3567/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					DuPont			</td>
			<td>
				
					DuPont Cellulosic Ethanol Demonstration plant				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3202/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					ECB Group			</td>
			<td>
				
					Omega Green Project				
			</td>
			<td>
					Paraguay                                            			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3907/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ECK			</td>
			<td>
				
					Krakow 3.4				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3686/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ECK			</td>
			<td>
				
					Krakow 1				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3684/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ECK			</td>
			<td>
				
					Krakow 2				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3685/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ECN			</td>
			<td>
				
					MILENA Gasifier				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3038/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ECO Biochemical Technology 			</td>
			<td>
				
					HVO plant				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4002/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ecoloop GmbH			</td>
			<td>
				
					Ecoloop GmbH				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4014/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EC Webrzeze			</td>
			<td>
				
					Gdansk 2,3,4,5				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3692/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EC Webrzeze			</td>
			<td>
				
					Gdynia unit #1.2				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3680/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EDF Energy			</td>
			<td>
				
					Cottam 2				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3683/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EDF Energy			</td>
			<td>
				
					Cottam 1,3,4				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3682/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EDF Energy			</td>
			<td>
				
					West Burton 1,2,3,4				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3681/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EDF Energy			</td>
			<td>
				
					Cottam				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3537/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EDF Energy			</td>
			<td>
				
					West Burton				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3526/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ekobenz			</td>
			<td>
				
					Bogumilow plant				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3870/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Electrabel (www.electrabel.be)			</td>
			<td>
				
					Rodenhuize				
			</td>
			<td>
					Belgium                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3693/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Electrabel (www.electrabel.be)			</td>
			<td>
				
					Ruien				
			</td>
			<td>
					Belgium                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3467/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Electrabel (www.electrabel.be)			</td>
			<td>
				
					Les Awirs				
			</td>
			<td>
					Belgium                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3694/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Elyse Energy			</td>
			<td>
				
					BioTJet				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4033/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Emamejeriet AB			</td>
			<td>
				
					Emamejeriet (Ema dairy)				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3785/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Emerald Biofuels			</td>
			<td>
				
					Emerald one				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3918/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					EMPA Duebendorf			</td>
			<td>
				
					CHP Duebendorf				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3064/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EnBW			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3482/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Endesa Italia			</td>
			<td>
				
					Monfalcone				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3584/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Endesa Italia			</td>
			<td>
				
					Monfalcone				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3583/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Endesa Italia			</td>
			<td>
				
					Fiumesanto				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3586/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ENEL			</td>
			<td>
				
					Sulcis 3				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3582/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ENEL			</td>
			<td>
				
					Sulcis 2				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3581/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ENEL			</td>
			<td>
				
					Genova				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3585/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Energia Uno			</td>
			<td>
				
					Urbas Terni				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3158/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Energie Oberwart			</td>
			<td>
				
					FICFB Oberwart				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3076/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Energi Randers			</td>
			<td>
				
					Randers Cogeneration plant				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3600/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Energy &amp; Chemical Department of East China University of Science and Technology			</td>
			<td>
				
					Energy pilot				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3358/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Energy Works			</td>
			<td>
				
					Energy Works Hull				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3972/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Enerkem			</td>
			<td>
				
					Westbury commercial demonstration facility				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3190/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Enerkem			</td>
			<td>
				
					Varennes Carbon Recycling				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3289/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Enerkem			</td>
			<td>
				
					Synthesis Enerkem Sherbrooke				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3039/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Enerkem			</td>
			<td>
				
					Varennes Carbon Recycling				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3289/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Enerkem			</td>
			<td>
				
					Synthesis Enerkem Sherbrooke				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3039/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Enerkem Alberta Biofuels LP			</td>
			<td>
				
					Edmonton Waste-to-Biofuels Project				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3182/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Enerkem, SA (Agbar), Repsol			</td>
			<td>
				
					Ecoplanta Molecular Recycling Solutions				
			</td>
			<td>
					Spain                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3865/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Engie + consortium			</td>
			<td>
				
					Gaya				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3304/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Eni			</td>
			<td>
				
					Biorefinery Gela				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3925/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Eni			</td>
			<td>
				
					Waste to Hydrogen				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3868/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Eni			</td>
			<td>
				
					SAF Livorno				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3994/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Eni			</td>
			<td>
				
					Biorefinery Gela				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3925/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Eni			</td>
			<td>
				
					SAF Livorno				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3994/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Eni 			</td>
			<td>
				
					Livorno Refinery				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4028/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ENI			</td>
			<td>
				
					Biorefinery Venice				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3313/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Eni SPA			</td>
			<td>
				
					Eni Taranto refinery				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3993/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ensyn			</td>
			<td>
				
					pilot				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3412/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ensyn			</td>
			<td>
				
					Cote Nord Project				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3710/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Ensyn			</td>
			<td>
				
					Aracruz Project				
			</td>
			<td>
					Brazil                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3413/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ensyn			</td>
			<td>
				
					C�te-Nord				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3815/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Ensyn and Renova Capital			</td>
			<td>
				
					GEORGIA PROJECT				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3896/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ensyn, Arbec Forest Products			</td>
			<td>
				
					C�TE NORD				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3891/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Envergent Technologies			</td>
			<td>
				
					Demoplant				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3728/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Eon			</td>
			<td>
				
					Ratcliffe				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3533/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Eon			</td>
			<td>
				
					Ironbridge				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3539/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Eon			</td>
			<td>
				
					Kingsnorth				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3527/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					E-ON Benelux			</td>
			<td>
				
					Maasvlakte 1+2				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3522/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					E.ON Gasification Development AB			</td>
			<td>
				
					Bio2G				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3116/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EPZ			</td>
			<td>
				
					Borssele 12				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3519/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ERSA			</td>
			<td>
				
					Rybnik 5,6,7,8				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3691/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ERSA			</td>
			<td>
				
					Rybnik 1,2,3,4				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3690/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Eska Graphic Board			</td>
			<td>
				
					Waste Paper Rejects Gasification				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3703/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Essent			</td>
			<td>
				
					Amercentrale 8				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3521/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Essent			</td>
			<td>
				
					Amercentrale 9				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3520/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Etel�-Savon Energia Oy			</td>
			<td>
				
					Pursiala CHP plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3650/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ettenberger Fulda			</td>
			<td>
				
					Ettenberger Fulda				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3745/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					euglena Co.,Ltd.			</td>
			<td>
				
					euglena demonstration (with NEDO)				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3952/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EWAG			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3476/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					EXYTRON GmbH			</td>
			<td>
				
					Exytron Demonstrationsanlage				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3956/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fechner GmbH			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3487/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Federated Co-operative 			</td>
			<td>
				
					Renewable diesel plant				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3998/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fernwaerme Neumarkt  Ges.m.b.H. &amp; Co.KG			</td>
			<td>
				
					CHP Urbas Neumarkt				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3101/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fintoil			</td>
			<td>
				
					Hamina CTO biorefinery				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3981/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					FLITE			</td>
			<td>
				
					LanzaJetTM Alcohol to Jet (AtJ) 				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3921/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Former American Process then GranBio			</td>
			<td>
				
					Alpena Biorefinery				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3713/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fortum Power and Heat AB			</td>
			<td>
				
					Hasselbyvaerket				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3516/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fortum Power and Heat Oy			</td>
			<td>
				
					Kauttua CHP plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3639/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fortum Power and Heat Oy			</td>
			<td>
				
					Kuusamo CHP plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3602/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fortum Power and Heat Oy			</td>
			<td>
				
					Sateri power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3675/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fortum Power and Heat Oy			</td>
			<td>
				
					Kokkola CHP plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3630/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fortum Power and Heat Oy			</td>
			<td>
				
					Naantali CHP plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3615/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fortum Power and Heat Oy			</td>
			<td>
				
					Lohja heating plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3610/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fortum Power and Heat Oy			</td>
			<td>
				
					Naantali CHP plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3651/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fortum Power and Heat Oy			</td>
			<td>
				
					Kontiosuo CHP plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3617/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fortum Power and Heat Oy			</td>
			<td>
				
					Lohja heating plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3648/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fraunhofer Umsicht			</td>
			<td>
				
					To-Syn-Fuel				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3959/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fraunhofer UMSICHT			</td>
			<td>
				
									
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3417/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Fraunhofer UMSICHT			</td>
			<td>
				
					Fraunhofer UMSICHT				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3416/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fraunhofer UMSICHT			</td>
			<td>
				
									
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3417/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Friedrich Wahl GmbH &amp; Co. KG			</td>
			<td>
				
					 CHP Urbas Sulzbach-Laufen				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3102/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Frontier Renewable Resources			</td>
			<td>
				
					Kinross Plant 1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3245/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Fulcrum Bioenergy			</td>
			<td>
				
					Trinity Fuels Plant - US Golf Coast				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4026/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Fulcrum Bioenergy			</td>
			<td>
				
					Centerpoint				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3915/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Fulcrum BioEnergy 			</td>
			<td>
				
					Sierra Biofuels				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3092/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Fulcrum BioEnergy and Essar Oil			</td>
			<td>
				
					Fulcrum NorthPoint				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3910/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Fulcrum (Sierra Biofuels)			</td>
			<td>
				
					Sierra				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3351/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Future Energy Resources (FERCO)			</td>
			<td>
				
					McNeil Generating Station				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3543/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					G4 Insight			</td>
			<td>
				
					PCH Alberta				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3826/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					G4 Insight			</td>
			<td>
				
					PCH Placer County				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3825/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Gevo			</td>
			<td>
				
					Silsbee				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3912/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Gevo			</td>
			<td>
				
					Gevo				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3283/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Gevo			</td>
			<td>
				
					Net-Zero 1 				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3924/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					GIDARA Energy B.V.			</td>
			<td>
				
					Advanced Methanol Amsterdam AMA				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3980/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Global Bioenergies			</td>
			<td>
				
					Isobutene demo				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3811/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Glock Energie GmbH			</td>
			<td>
				
					Griffen CHP 				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3856/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Graebner Esslingen			</td>
			<td>
				
					Graebner Esslingen				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3751/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Graebner Hochsauerland			</td>
			<td>
				
					Graebner Hochsauerland				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3746/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Graebner Rosenheim			</td>
			<td>
				
					Graebner Rosenheim				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3756/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					GranBio			</td>
			<td>
				
					Bioflex 1				
			</td>
			<td>
					Brazil                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3269/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					GranBio Technologies			</td>
			<td>
				
					Thomaston GP+ Biorefinery				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3715/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Greenergy			</td>
			<td>
				
					Advanced biofuels project				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3968/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Greenfield Ethanol			</td>
			<td>
				
					Greenfield				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3343/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					GreenflexJET Consortium			</td>
			<td>
				
					GreenflexJET Demo				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3965/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Green Fuel Nordic			</td>
			<td>
				
					GNF Lieksa				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3848/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Green Fuel Nordic			</td>
			<td>
				
					Green Fuel Nordic 				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3300/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					GRESCO Power Solution GmbH			</td>
			<td>
				
					HGKW Bad Wildungen				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3885/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					GRETHA			</td>
			<td>
				
					Nongbua DFB gasifier				
			</td>
			<td>
					Thailand                                            			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3791/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Groupe Onym			</td>
			<td>
				
					Parent Demonstration Plant				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3403/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					GTI Gas Technology Institute			</td>
			<td>
				
					GTI gasifier Des Plaines				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3041/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Guascor Italia			</td>
			<td>
				
					Rossano Calabro (CS)				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3129/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Guessing Renewable Energy			</td>
			<td>
				
					FICFB Guessing				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3059/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					H2Herten GmbH			</td>
			<td>
				
					Blue Tower Technology Herten				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3069/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					HCS Group and Gevo			</td>
			<td>
				
					SAF Speyer				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4007/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					HEH Holzenergie			</td>
			<td>
				
					CHP Pfalzfeld				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3070/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Heizkraftwerk W�rzburg			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3497/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Henan Tianguan Group			</td>
			<td>
				
					Henan 1				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3354/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Henan Tianguan Group			</td>
			<td>
				
					Henan 2				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3707/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					HGKW Bad Wildungen GmbH			</td>
			<td>
				
					HGKW Bad Wildungen GmbH				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3765/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					H.H. Kaeser GmbH			</td>
			<td>
				
					Holzgasanlage 1 Kaeser Gasel 				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3699/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					H.H. Kaeser GmbH			</td>
			<td>
				
					Holzgasanlage 2 Kaeser Gasel 				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3700/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Hindustan Petroleum			</td>
			<td>
				
					2G ethanol Commercial plant				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3946/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					HKW II (Stadtwerke Duisburg)			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3479/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					HOFOR			</td>
			<td>
				
					Amager BIO4				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3768/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Holzstrom aus Nidwalden			</td>
			<td>
				
					CHP Pyroforce Nidwalden				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3062/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Holzstrom GmbH			</td>
			<td>
				
					CHP Urbas Neukirchen				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3084/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Honeywell UOP and Preem			</td>
			<td>
				
					Co-processing trial				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3987/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					HoSt			</td>
			<td>
				
					CFB Tzum				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3083/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Hotel Haffhus			</td>
			<td>
				
					Hotel Haffhus				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3853/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					HS Energieanlagen GmbH			</td>
			<td>
				
					CHP Heatpipe Reformer Neufahrn bei Freising				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3066/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					IBN-One (JV of Cristal Union and Global Bioenergies)			</td>
			<td>
				
					Isobutene commercial				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3879/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ICQ/SIAG/ERBA			</td>
			<td>
				
					-				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3139/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					IES Utilities Inc			</td>
			<td>
				
					Ottumwa Generating Station #1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3559/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					IFP			</td>
			<td>
				
					Futurol				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3830/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Illinois Power Company (IP)			</td>
			<td>
				
					Vermilion Power Station #1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3566/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ilomantsin Laempoe Oy			</td>
			<td>
				
					District heating plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3175/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Imperial Oil			</td>
			<td>
				
					Strathcona renewable diesel project				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3984/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Indaba Renewable Fuels			</td>
			<td>
				
					Indaba Renewable Fuels Refinery II				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4005/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Indaba Renewable Fuels			</td>
			<td>
				
					Indaba Renewable Fuels Refinery				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4004/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Indian Glycol &amp; DBT-ICT Mumbai			</td>
			<td>
				
					2G ethanol demo plant				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3943/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Indian Oil Corporation			</td>
			<td>
				
					3G ethanol Commercial plant				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3929/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Indian Oil Corporation			</td>
			<td>
				
					2G ethanol Commercial plant				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3803/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Indian Oil Corporation			</td>
			<td>
				
					2G ethanol demo plant				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3930/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Indian Oil R&amp;D			</td>
			<td>
				
					2G ethanol Pilot plant				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3931/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Ineos Bio			</td>
			<td>
				
					Ineos Bio UK				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3326/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Institute of Energy Process Engineering and Chemical Engineering (IEC), TU Bergakademie Freiberg			</td>
			<td>
				
					FlexiSlag Pilot Plant				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4020/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					International Power			</td>
			<td>
				
					Rugeley				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3534/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Iogen Corporation			</td>
			<td>
				
					demo				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3192/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Iowa State University			</td>
			<td>
				
					BioCentury Research Farm				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3208/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Iowa State University			</td>
			<td>
				
									
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3418/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Isar Amperewerke / Bayernwerk			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3498/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Jalasjaerven Laempoe Oy			</td>
			<td>
				
					District heating				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3172/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Japanese Consortium			</td>
			<td>
				
					NEDO UCO to SAF Project				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3982/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					J�rvi-Suomen Voima Oy			</td>
			<td>
				
					Ristiina power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3665/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					J�rvi-Suomen Voima Oy			</td>
			<td>
				
					Savonlinna power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3669/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Jilin Fuel Alcohol			</td>
			<td>
				
					Jilin 2				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3356/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Joint Venture of Air Liquide, Enerkem, Port of Rotterdam and Shell			</td>
			<td>
				
					W2C - Waste to jet				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3847/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Josef Bucher AG Escholzmatt			</td>
			<td>
				
					Holzverstromungsanlage Bucher Escholzmatt				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3702/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Jura BHKW Hiller			</td>
			<td>
				
					Jura BHKW Hiller				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3855/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					JV controlled by ENI			</td>
			<td>
				
					Waste to Methanol				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3908/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Jyv�skyl�n Energiantuotanto Oy			</td>
			<td>
				
					Rauhalahti CHP plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3619/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Jyv�skyl�n Voima Oy			</td>
			<td>
				
					Keljonlahti				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3620/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kainuun Voima Oy			</td>
			<td>
				
					CHP plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3622/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kansas City (MO) Power &amp; Light			</td>
			<td>
				
					La Cygne Generating Station #1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3555/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Kanteleen Voima			</td>
			<td>
				
					Nordfuel biorefinery				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3860/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kanteleen Voima Oy			</td>
			<td>
				
					Haapavesi power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3637/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Karlsruhe Institute of Technology			</td>
			<td>
				
					KIT PDU				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3441/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Karlsruhe Institute of Technology (KIT)			</td>
			<td>
				
					bioliq				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3240/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Karlsruhe Institute of Technology (KIT)			</td>
			<td>
				
					bioliq				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3442/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Karlsruhe Institute of Technology (KIT)			</td>
			<td>
				
					bioliq				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3240/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Karlsruhe Institute of Technology (KIT)			</td>
			<td>
				
					bioliq				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3932/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Karlsruhe Institute of Technology (KIT)			</td>
			<td>
				
					bioliq				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3442/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Karlsruhe Institute of Technology (KIT)			</td>
			<td>
				
					bioliq				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3932/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kauhajoen Laempoehuolto Oy			</td>
			<td>
				
					District heating plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3174/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kaukaan Voima Oy			</td>
			<td>
				
					Lappeenranta power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3646/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Keravan L�mp�voima Oy			</td>
			<td>
				
					Kerava L�mp�voima				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3626/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kerry			</td>
			<td>
				
					Ontario Facility				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3411/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					KEW Technology Ltd.			</td>
			<td>
				
					Sustainable Energy Centre				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3967/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					KiOR			</td>
			<td>
				
					KiOR				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3337/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					KiOR			</td>
			<td>
				
					Columbus I				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3421/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kiteen Laempoe Oy			</td>
			<td>
				
					District heating plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3173/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kogeneracja			</td>
			<td>
				
					Wroclaw BC-1				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3687/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kogeneracja			</td>
			<td>
				
					Wroclaw BC-2 BC-3				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3688/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kogeneracja			</td>
			<td>
				
					Czechnica K2				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3689/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kokemaen Laempoe Oy			</td>
			<td>
				
					CHP power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3177/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kokkolan Voima Oy			</td>
			<td>
				
					Kokkola CHP plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3629/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Kopf Syngas GmbH and Co. KG			</td>
			<td>
				
					KSV Mannheim				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3757/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kotkan Energia Oy			</td>
			<td>
				
					Hovinsaari power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3628/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kuhmon Lampo Oy			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3605/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kungliga Tekniska Hoegskolan (Royal Institute of Technology)			</td>
			<td>
				
					Centre for Direct Gasification of Biomass				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3125/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kuntschar Wolfshagen			</td>
			<td>
				
					Kuntschar Wolfshagen				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3748/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kuopion Energia			</td>
			<td>
				
					Haapaniemi power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3606/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kuopion Energia			</td>
			<td>
				
					Haapaniemi Two				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3603/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					KWS Landshut			</td>
			<td>
				
					KWS Landshut				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3752/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					KWS Ostalb			</td>
			<td>
				
					KWS Ostalb				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3761/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Kymin Voima Oy			</td>
			<td>
				
					Kuusankoski power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3643/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Laanilan Voima Oy			</td>
			<td>
				
					Oulu mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3653/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Lahti Energia Oy			</td>
			<td>
				
					Heinola power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3632/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Lahti Energia Oy			</td>
			<td>
				
					Kymijarvi power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3644/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Lahti Energia Oy			</td>
			<td>
				
					Kymijaervi II				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3167/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Lahti Energia Oy			</td>
			<td>
				
					Kymijaervi I				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3166/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Lahti Energia Oy			</td>
			<td>
				
					Kymijaervi II				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3167/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Lakeland Electric			</td>
			<td>
				
					Lakeland Electric #3				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3557/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Lamprecht			</td>
			<td>
				
					Lamprecht GmH				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3443/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					LanzaJet			</td>
			<td>
				
					Freedom Pines				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3920/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					LanzaTech			</td>
			<td>
				
					USA Mobile Demo Plant				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3277/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					LanzaTech - Concord Enviro Systems PVT Ltd. 			</td>
			<td>
				
					MSW Syngas to Electricity and Fuel				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3263/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					LanzaTech UK			</td>
			<td>
				
					DRAGON Decarbonizing and Reimagining Aviation for the Goal Of Netzero				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3976/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Lenzing AG			</td>
			<td>
				
					Lenzing				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3464/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Levenseat Renewable Energy Ltd			</td>
			<td>
				
					Levenseat EfW 				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3971/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Licella			</td>
			<td>
				
					Commercial demonstration plant				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3264/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Licella			</td>
			<td>
				
					Commercial demonstration plant plastics				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3928/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					 Licella Pty Ltd			</td>
			<td>
				
					LICELLA ? NEW SOUTH WALES				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3897/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ligento Nuernberg			</td>
			<td>
				
					Ligento Nuernberg				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3760/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Lignol			</td>
			<td>
				
					Demo				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3290/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Liquid Wind / Orsted			</td>
			<td>
				
					FlagshipONE				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3991/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Longlive Bio-technology Co. Ltd.			</td>
			<td>
				
					Longlive				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3278/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					LTU Green Fuels			</td>
			<td>
				
					DP1+DME pilot				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3048/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Macquarie Generation			</td>
			<td>
				
					Liddell				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3457/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Madison Gas and Electric Company			</td>
			<td>
				
					Blount Street				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3558/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Mainstream Engineering Corporation			</td>
			<td>
				
									
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3422/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					M�nt�n Energia Oy			</td>
			<td>
				
					Mantta power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3649/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Marathon Petroleum			</td>
			<td>
				
					Dickinson Renewable Diesel Facility				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3988/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Marathon Petroleum and Neste (Joint venture)			</td>
			<td>
				
					Martinez Renewable Fuels Facility				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3989/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Mascoma			</td>
			<td>
				
					commercial				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3291/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					MASH ENERGY			</td>
			<td>
				
					MASH ENERGY				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3902/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Mekrijaervi Research Station			</td>
			<td>
				
					District heating plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3176/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Metso Fibre			</td>
			<td>
				
					Bioproduct Mill Aanekoski				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3796/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Metso Fibre Oy, Joutseno Mill			</td>
			<td>
				
					Lime kiln gasifier				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3164/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					MEVA Innovation			</td>
			<td>
				
					VIPP Demonstration				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3121/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Mississippi State University			</td>
			<td>
				
									
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3423/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Mondi Papier &amp; Zellstoff			</td>
			<td>
				
					Mondi Papier &amp; Zellstoff				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3463/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					M-real Oyj			</td>
			<td>
				
					Simpele mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3664/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Muensterland Energy Gmbh			</td>
			<td>
				
					Muensterland Energy Gmbh				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3758/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Myllykoski Paper Oy			</td>
			<td>
				
					Myllykoski papermill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3642/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Georgia Power				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3552/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Avesta Energiverk				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3503/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Vasthamnsvert CHP				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3505/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Karlstad Energiverken				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3506/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					PT Indah Kiat Pulp &amp; Paper				
			</td>
			<td>
					Indonesia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3451/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Ocean Sky Co				
			</td>
			<td>
					Indonesia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3450/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3621/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Nykopping Energy				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3512/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Soderenergi AB				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3515/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					VEAG				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3488/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Hunosa power station				
			</td>
			<td>
					Spain                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3501/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Bayernwerke AG				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3473/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Outokumpo Oy				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3655/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3477/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3472/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					SEPCO				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3574/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Spain                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3502/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					n.a.			</td>
			<td>
				
					Saarberwerke AG				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3491/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					National Renewable Energy Laboratory			</td>
			<td>
				
									
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3424/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Naturenergie Hersbruck GmbH &amp; Co. KG			</td>
			<td>
				
					Naturenergie Hersbruck GmbH &amp; Co. KG				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3759/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Neckarwerke			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3494/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Neste			</td>
			<td>
				
					Tall oil pitch				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3724/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Neste			</td>
			<td>
				
					Singapore expansion				
			</td>
			<td>
					Singapore                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3845/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Neste			</td>
			<td>
				
					Singapore				
			</td>
			<td>
					Singapore                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3185/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Neste			</td>
			<td>
				
					Modify Rotterdam refinery for SAF				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3978/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Neste			</td>
			<td>
				
					Porvoo 2				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3183/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Neste			</td>
			<td>
				
					Rotterdam				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3184/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Neste			</td>
			<td>
				
					Porvoo 1				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3195/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Neste			</td>
			<td>
				
					Rotterdam				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3184/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Neste			</td>
			<td>
				
					Rotterdam expansion				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3914/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Neste			</td>
			<td>
				
					Porvoo 2				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3183/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Neste			</td>
			<td>
				
					Porvoo 1				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3195/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Neste			</td>
			<td>
				
					Rotterdam expansion				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3914/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Neue Energy Premnitz			</td>
			<td>
				
					Premnitz project				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4016/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					New York State Electric and Gas (NYSEG)			</td>
			<td>
				
					Greenidge Generating Station #6				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3546/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Niagara Mohawk Power Corp.			</td>
			<td>
				
					Dunkirk Steam Station #1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3547/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					NIOSH (National Institute Occupational Safety and Health)			</td>
			<td>
				
					National Institute of Occupational Safety and Health (NIOSH)				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3569/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					NIPSCO			</td>
			<td>
				
					Bailey Generating Station #7				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3544/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Nordic Electrofuel AS			</td>
			<td>
				
					E-fuel 1				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3973/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Norrkopings Kraft			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3510/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					North Dakota Dept. of Corrections and Rehabilitation			</td>
			<td>
				
					North Dakota State Penitentiary				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3542/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Northern Indiana Public Service Company (NIPSCO)			</td>
			<td>
				
					Michigan City Generating Station #12				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3556/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Northern States Power			</td>
			<td>
				
					King (Allen S.) Generating Station #1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3576/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Northern States Power Company			</td>
			<td>
				
					BL Station #1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3549/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Northern States Power Company			</td>
			<td>
				
					Bay Front Station				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3541/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					NREL (National Renewable Energy Laboratory)			</td>
			<td>
				
					Integrated Biorefinery Research Facility (IBRF)				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3271/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Nukoping Energiverk			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3511/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Nuon			</td>
			<td>
				
					Hemweg Centrale (HW8)				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3518/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					NUON/Vattenfall			</td>
			<td>
				
					Wood co-gasification in IGCC				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3082/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Nurmes			</td>
			<td>
				
					Micro-scale biomass gasification CHP Volter				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3170/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					OKI Pulp and Paper Mill / APP			</td>
			<td>
				
					OKI				
			</td>
			<td>
					Indonesia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3105/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					OMV			</td>
			<td>
				
					Co-processing of vegetable oils				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3909/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					ONYM Group			</td>
			<td>
				
					Montreal Technology Showcase Plant				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4027/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					OPG			</td>
			<td>
				
					Nanticoke 4				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3589/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					OPG			</td>
			<td>
				
					Thunder Bay 2				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3592/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					OPG			</td>
			<td>
				
					Lambton 1				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3593/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					OPG			</td>
			<td>
				
					Nanticoke X				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3594/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					OPG			</td>
			<td>
				
					Nanticoke 6				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3590/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					OPG			</td>
			<td>
				
					Atikokan				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3588/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					OPG			</td>
			<td>
				
					Thunder Bay 3				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3591/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					ORLEN Poludnie (part of ORLEN GROUP)			</td>
			<td>
				
					Jedlicze Site				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3862/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Ostersunds Fjarrvarme			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3513/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Otter Tail Power Co.			</td>
			<td>
				
					Big Stone Plant #1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3561/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Oulun Energia			</td>
			<td>
				
					Toppila power plants				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3652/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Oy Alholmens Kraft Ab			</td>
			<td>
				
					Pietarsaari power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3657/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Oy Mets�-Botnia Ab			</td>
			<td>
				
					Kemi mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3624/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Oy Turku Energia Ab			</td>
			<td>
				
					Linnankatu power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3609/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Oy Turku Energia Ab			</td>
			<td>
				
					Linnankatu power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3673/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Pertamina			</td>
			<td>
				
					Pertamina refinery 				
			</td>
			<td>
					Indonesia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4003/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					P.H.Glatfelter Co			</td>
			<td>
				
					Spring Grove Paper Mill				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3575/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Phillips 66			</td>
			<td>
				
					Rodeo Renewed				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3919/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Pittsburgh Brewing Company			</td>
			<td>
				
					Pittsburgh Brewing Company				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3570/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					PKN ORLEN			</td>
			<td>
				
					Plock Plant				
			</td>
			<td>
					Poland                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4029/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					POET			</td>
			<td>
				
					Scotland				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3220/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					POET-DSM Advanced Biofuels			</td>
			<td>
				
					Project Liberty				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3196/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					PoliTO			</td>
			<td>
				
					Wood Gasifier				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3135/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Pond Biofuels			</td>
			<td>
				
					Pond Biofuels				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3330/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Pori Energia Oy			</td>
			<td>
				
					Aittaluoto power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3659/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Porin Prosessivoima Oy			</td>
			<td>
				
					Pori mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3611/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Porin Prosessivoima Oy			</td>
			<td>
				
					Pori mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3660/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Porin Prosessivoima Oy			</td>
			<td>
				
					Pori mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3658/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Powerflute Oy			</td>
			<td>
				
					Savon Sellu				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3604/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Praj Industries			</td>
			<td>
				
					2G ethanol demo plant				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3944/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Preem			</td>
			<td>
				
					Preem HVO 2015				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3318/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Preem			</td>
			<td>
				
					Revamp Synsat unit				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3977/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Preem			</td>
			<td>
				
					Preem HVO diesel and biojet				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3838/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Preussen Elektra			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3480/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Pritscher Landshut			</td>
			<td>
				
					Pritscher Landshut				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3753/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Provincial Electricity Authority of Thailand			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Thailand                                            			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3453/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Pyrocell AB			</td>
			<td>
				
					Pyrocell				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3895/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Pyrocell (JV of Setra and Preem)			</td>
			<td>
				
					pyrolysis oil upgrading				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3827/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Pyroneer - Orsted A/S			</td>
			<td>
				
					Pyroneer Demonstration Plant				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3114/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Qalovis Altenberge			</td>
			<td>
				
					Qalovis				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3737/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Qualovis Altenberge			</td>
			<td>
				
					Qualovis Altenberge				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3736/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Queensland University of Technology			</td>
			<td>
				
					Mackay Renewable Biocommodities Pilot Plant				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3213/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Raizen Energia			</td>
			<td>
				
					Brazil				
			</td>
			<td>
					Brazil                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3286/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Rau GmbH			</td>
			<td>
				
					Urbas Balingen				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3156/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Rauman Voima Oy			</td>
			<td>
				
					Rauman Voima Oy				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3662/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Red Arrow			</td>
			<td>
				
					Red Arrow				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3428/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Red Arrow			</td>
			<td>
				
					Red Arrow				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3429/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Red Arrow			</td>
			<td>
				
					Red Arrow				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3426/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Red Rock Biofuels			</td>
			<td>
				
					Commercial				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3731/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RegaWatt			</td>
			<td>
				
					RegaWatt Abensberg				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3735/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					REG Geismar			</td>
			<td>
				
					REG Geismar Expansion				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3926/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					REG Geismar			</td>
			<td>
				
					Geismar Project				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3238/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Reliant Energy			</td>
			<td>
				
					Shawville Generating Station #3				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3553/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Reliant Energy			</td>
			<td>
				
					Shawville Generating Station #2				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3554/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Reliant Energy			</td>
			<td>
				
					Seward Generating Station #12				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3568/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Renewable Oil International LLC			</td>
			<td>
				
									
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3430/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					ReNew ELP			</td>
			<td>
				
					Plastic recycling plant				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3849/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					RenFuel			</td>
			<td>
				
					RenFuel				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3843/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RenFuel			</td>
			<td>
				
					RenFuel Backhammer				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3829/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Renmatix			</td>
			<td>
				
					Demonstration Plant				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3244/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Repsol			</td>
			<td>
				
					Low Emissions Advanced biofuels plant				
			</td>
			<td>
					Spain                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3966/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Rheinbraun AG			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3474/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Rheinhafendampfkraftwerk			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3484/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RISE ETC			</td>
			<td>
				
					PEGB Pilot, FOX				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3786/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RISE ETC			</td>
			<td>
				
					Pilot				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3859/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RISE ETC			</td>
			<td>
				
					PEGB Pilot, FOX				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3786/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RISE ETC			</td>
			<td>
				
					Pilot				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3859/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RiverRidge			</td>
			<td>
				
					Full Circle Energy Facility				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3888/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Romande Energie			</td>
			<td>
				
					Puidoux Woodgasifier				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3795/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Romande Energie Services SA			</td>
			<td>
				
					Kombi Power System Charmey				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3858/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Rottneros			</td>
			<td>
				
					Rottneros Biorefinery				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3722/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Rottneros			</td>
			<td>
				
					Vallvik Biofuels				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3723/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Rottneros			</td>
			<td>
				
					Rottneros Biorefinery				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3722/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Rottneros			</td>
			<td>
				
					Vallvik Biofuels				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3723/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Rovaniemen Energia Oy			</td>
			<td>
				
					Suosiola power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3666/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RTI International			</td>
			<td>
				
					RTI International				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3431/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Rumford Cogen Co.			</td>
			<td>
				
					Rumford Cogen Co.				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3573/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RWE			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3496/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					RWE Essent			</td>
			<td>
				
					Wood gasifier Geertruidenberg				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3060/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RWEnpower			</td>
			<td>
				
					Aberthaw				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3535/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RWEnpower			</td>
			<td>
				
					Tilbury				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3587/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					RWEnpower			</td>
			<td>
				
					Didcot				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3538/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					RWE Power AG			</td>
			<td>
				
					MFC within ITZ-CC				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3851/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					RWE Power AG			</td>
			<td>
				
					MFC within ITZ-CC				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3852/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					RWE Power AG			</td>
			<td>
				
					MFC within ITZ-CC				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3851/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					RYAM Rayonier Advanced Materials Inc.			</td>
			<td>
				
					2G bioethanol refinery 				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4011/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SaarEnergie AG			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3495/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Sainc Energy Limited			</td>
			<td>
				
					Cordoba				
			</td>
			<td>
					Spain                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3814/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Saint-Gobain France			</td>
			<td>
				
					Veralia Champagne (Xylowatt)				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3882/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Sande Paper Mill A/S			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3500/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Santee Cooper			</td>
			<td>
				
					Jefferies Generating Station #3 and #4				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3563/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Savon Voima			</td>
			<td>
				
					Joensuu Facility				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3415/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Savon Voima L�mp� Oy			</td>
			<td>
				
					Iisalmi power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3631/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Savon Voima L�mp� Oy			</td>
			<td>
				
					Pieksamaki power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3656/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					SCA			</td>
			<td>
				
					SCA Biorefinery Ostrand AB				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3942/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Scottish and Southern Energy			</td>
			<td>
				
					Fiddlers Ferry				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3532/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Scottish and Southern Energy			</td>
			<td>
				
					Ferrybridge				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3531/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Scottish and Southern Energy			</td>
			<td>
				
					Slough Heat and Power Ltd.				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3525/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Scottish Power			</td>
			<td>
				
					Longannet				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3528/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Scottish Power			</td>
			<td>
				
					Cockenzie				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3536/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Seaboard Energy			</td>
			<td>
				
					Seaboard Energy Hugorton				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4025/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Sekab Group			</td>
			<td>
				
					Biorefinery Demo Plant				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3228/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Sekisui Chemical Co.,Ltd.			</td>
			<td>
				
					Ethanolization of garbage				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3836/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					SEK Koblenz			</td>
			<td>
				
					KSV Koblenz				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3747/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Shandong Zesheng Biotech Co.			</td>
			<td>
				
					Zesheng				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3357/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Shanxi Yingjiliang Biomass Company and Shanghai Jiao Tong University			</td>
			<td>
				
					DALI COUNTY FACILITY				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3899/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Shell			</td>
			<td>
				
					Rotterdam biofuels facility				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3986/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Shell 			</td>
			<td>
				
					IH2 Drop-in fuel Demo plant				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3824/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Shell Catalysts &amp; Technologies			</td>
			<td>
				
					IH2 DEMONSTRATION FACILITY 				
			</td>
			<td>
					India                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3900/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SILVA GREEN FUEL 			</td>
			<td>
				
					SILVA GREEN FUEL 				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3903/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Silva Green Fuel (JV of Statkraft and Sodra)			</td>
			<td>
				
					industrial scale demo				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3800/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Sindal District Heating Company			</td>
			<td>
				
					Dall Energy CHP plant in Sindal 				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3788/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Skelleftea Kraft			</td>
			<td>
				
					Hedesbyn				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3514/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Skive District Heating Company			</td>
			<td>
				
					Skive CHP plant				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3075/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					SkyNRG			</td>
			<td>
				
					DSL-01				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3846/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Sodra			</td>
			<td>
				
					Sodra biomethanol				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3869/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Solena Fuels			</td>
			<td>
				
					Solena UK				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3322/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Solvay Osterreich			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3462/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Southeast Paper			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3548/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Southern Company/Alabama Power Company			</td>
			<td>
				
					Gadsden Steam Plant #2				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3551/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Southern Company/Georgia Power Company			</td>
			<td>
				
					Harlee Branch Generating Station				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3562/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Southern Company/Georgia Power Company			</td>
			<td>
				
					Hammond Generating Station #1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3545/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Southern Company/Savannah Electric and Power Company (SEPCO)			</td>
			<td>
				
					Kraft / Riverside Plants #2				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3571/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Spanner Bamberg			</td>
			<td>
				
					Spanner Bamberg				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3750/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Spanner Landshut			</td>
			<td>
				
					Spanner Landshut				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3754/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Special Purpose Vehicule (MGGE)			</td>
			<td>
				
					Mont-Godinne Hospital (Xylowatt)				
			</td>
			<td>
					Belgium                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3881/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					St1			</td>
			<td>
				
					Cellulonix Follum				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3864/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					St1			</td>
			<td>
				
					Cellulonix Kajaani 2				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3877/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					St1			</td>
			<td>
				
					Etanolix Gothenburg				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3876/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					St1			</td>
			<td>
				
					Cellulonix Kajaani				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3301/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					St1			</td>
			<td>
				
					Etanolix Hamina				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3872/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					St1			</td>
			<td>
				
					Etanolix Lahti				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3873/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					St1			</td>
			<td>
				
					Etanolix Vantaa				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3871/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					St1			</td>
			<td>
				
					Bionolix Hameenlinna				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3875/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					St1			</td>
			<td>
				
					Cellulonix Pietarsaari				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3878/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					St1			</td>
			<td>
				
					Etanolix Gothenburg				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3876/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					St1			</td>
			<td>
				
					Cellulonix Kajaani				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3301/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					St1			</td>
			<td>
				
					Cellulonix Follum				
			</td>
			<td>
					Norway                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3864/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					St1			</td>
			<td>
				
					Cellulonix Kajaani 2				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3877/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					St1			</td>
			<td>
				
					Cellulonix Pietarsaari				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3878/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					St1 and SCA in joint venture			</td>
			<td>
				
					St1 Gothenburg Biorefinery				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3842/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stadtwerke D?sseldorf			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3485/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					Stadtwerke Duesseldorf			</td>
			<td>
				
					CHP Arnsberg-Wildhausen				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3071/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					STADTWERKE KONSTANZ GmbH			</td>
			<td>
				
					CHP Urbas Konstanz				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3103/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stadtwerke Pforzheim			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3490/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stadtwerke Rosenheim GmbH			</td>
			<td>
				
					CHP Stadtwerke Rosenheim				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3697/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stadtwerke Rosenheim GmbH			</td>
			<td>
				
					CHP Stadtwerke Rosenheim				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3698/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stadtwerke Rosenheim GmbH			</td>
			<td>
				
					CHP Stadtwerke Rosenheim				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3696/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stadtwerke Rosenheim GmbH			</td>
			<td>
				
					CHP Stadtwerke Rosenheim				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3698/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Stanwell Corporation			</td>
			<td>
				
					Stanwell				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3460/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					STEAG AG			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3493/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Steeper Energy			</td>
			<td>
				
					CBS (Continuous Bench Scale)				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3293/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Steiner A. &amp; Cie AG			</td>
			<td>
				
					-				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3140/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stiesdal 			</td>
			<td>
				
					SkyClean				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3941/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Stirling DK			</td>
			<td>
				
					CHP Flensburg				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3109/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stirling DK			</td>
			<td>
				
					Carlow				
			</td>
			<td>
					Ireland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3112/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso			</td>
			<td>
				
					Lime kiln gasifier				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3168/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso			</td>
			<td>
				
					Gasifier at Varkaus paper mill (former Corenso)				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3053/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso			</td>
			<td>
				
					Lime kiln gasifier				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3168/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso Ltd			</td>
			<td>
				
					Stora Enso Fors Mill				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3504/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso Oyj			</td>
			<td>
				
					Varkaus mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3678/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso Oyj			</td>
			<td>
				
					Oulu mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3654/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso Oyj			</td>
			<td>
				
					Heinola Flutingmill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3633/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso Oyj			</td>
			<td>
				
					Imatra mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3607/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso Oyj			</td>
			<td>
				
					Veitsiluoto mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3625/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso Publication Papers Oy Ltd			</td>
			<td>
				
					Anjalankoski mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3640/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Stora Enso Publication Papers Oy Ltd			</td>
			<td>
				
					Summa mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3634/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					STORE&amp;GO Projektpartner			</td>
			<td>
				
					STORE&amp;GO Demonstrationsanlage Falkenhagen				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3953/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Strategic Biofuels LLC			</td>
			<td>
				
					Louisiana Green Fuels Project (LGF)				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4012/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					sunfire GmbH			</td>
			<td>
				
					sunfire Power-to-Liquids				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3957/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SunPine			</td>
			<td>
				
					SunPine HVO addition				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3841/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SunPine			</td>
			<td>
				
					SunPine HVO 100 mio litres				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3840/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SunPine			</td>
			<td>
				
					SunPine HVO addition				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3841/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Sunshine Kaidi			</td>
			<td>
				
					Kemi biorefinery				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3797/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Surrey Municipality			</td>
			<td>
				
					Surrey Biofuel facility				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3810/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Swedish Biofuels AB			</td>
			<td>
				
					Swedish Biofuels				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3962/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SynCraft			</td>
			<td>
				
					CraftWerk Innsbruck				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3781/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SynCraft			</td>
			<td>
				
					CraftWerk Vierschach				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3779/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SynCraft			</td>
			<td>
				
					CraftWerk Stadl				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3782/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SynCraft			</td>
			<td>
				
					CraftWerk Innsbruck				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3781/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SynCraft			</td>
			<td>
				
					CraftWerk Stadl				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3782/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SynCraft			</td>
			<td>
				
					CraftWerk Schwaz				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3778/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SynCraft			</td>
			<td>
				
					CraftWerk Vierschach				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3779/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SynCraft			</td>
			<td>
				
					CraftWerk Schwaz				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3778/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					SynCraft			</td>
			<td>
				
					CraftWerk Hatlerdorf				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3780/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Synkero			</td>
			<td>
				
					Commercial synthetic kerosene facility				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4000/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Tacoma Public Utilities			</td>
			<td>
				
					City Of Tacoma Steam Plant No. 2				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3577/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Tampa Electric Company (TECO)			</td>
			<td>
				
					Gannon (F.J.) Generating Station #3				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3578/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Tampereen S�hk�laitos			</td>
			<td>
				
					Naistenlahti power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3671/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Tarong Energy			</td>
			<td>
				
					Tarong				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3461/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Technical University of Munich			</td>
			<td>
				
					NAMOSYN - OME35 plant				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3958/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Technische Universtit�t Darmstadt			</td>
			<td>
				
					CLARA				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3933/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Tekniska Verken Ltd			</td>
			<td>
				
					Tekniska Verken Ltd 2				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3508/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Tekniska Verken Ltd			</td>
			<td>
				
					Tekniska Verken Ltd 1				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3507/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Tembec Chemical Group			</td>
			<td>
				
					Synthesis Tembec Chemical Quebec				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3050/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Thermochem Recovery International			</td>
			<td>
				
					technology development laboratory and pilot plant - thermochemical				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3216/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ThermoChem Recovery International (TRI)			</td>
			<td>
				
					Fully Integrated BioRefinery				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3787/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Tidewater 			</td>
			<td>
				
					Tidewater Prince George refinery				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3979/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Topell Nederland			</td>
			<td>
				
					Topell Nederland				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3792/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Tornion Voima Oy			</td>
			<td>
				
					Tornio power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3672/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Total			</td>
			<td>
				
					BioTfueL demo				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3766/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Total			</td>
			<td>
				
					La Mede				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3828/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Total			</td>
			<td>
				
					Grandpuits				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3911/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Total			</td>
			<td>
				
					BioTfueL demo				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3766/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Total			</td>
			<td>
				
					Grandpuits				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3911/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Total			</td>
			<td>
				
					La Mede				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3828/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Total			</td>
			<td>
				
					BioTfueL demo				
			</td>
			<td>
					France                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3766/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Tournai city			</td>
			<td>
				
					Tournai Swimming Pool (Xylowatt)				
			</td>
			<td>
					Belgium                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3884/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Toyo and Velocys			</td>
			<td>
				
					Biomass-to-jet fuel project				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3975/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Tri-State Generating &amp; Transmission Association, Inc.			</td>
			<td>
				
					Escalante Generating Station #1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3572/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					TUBITAK			</td>
			<td>
				
					TRIJEN (Liquid Fuel Production From Biomass and Coal Blends)				
			</td>
			<td>
					Turkey                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3246/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					TU Dresden			</td>
			<td>
				
					High-temperature reactor according to the principle of circulating fluidized bed 				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4018/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					TU Dresden			</td>
			<td>
				
					TC2 process				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4017/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					TU Freiberg			</td>
			<td>
				
					FlexiCOORVED Pilot Plant				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4019/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					TU Munich			</td>
			<td>
				
					Booster				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4021/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Turku energia and Gasek Oy			</td>
			<td>
				
					Wood gasification facility to generate steam for industrial  laundry in Turku				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3171/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					TVA			</td>
			<td>
				
					Allen (T.H) Fossil Plant				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3560/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					TVA			</td>
			<td>
				
					Colbert Fossil Plant #1				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3580/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					TVA			</td>
			<td>
				
					Kingston Fossil Plant #5				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3565/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Twence			</td>
			<td>
				
					Hengelo				
			</td>
			<td>
					Netherlands                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3819/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					UDG Niagara Goodyear			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3564/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					UDT			</td>
			<td>
				
									
			</td>
			<td>
					Chile                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3433/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Uniper			</td>
			<td>
				
					SkyFuel H2				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4032/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Uni Stuttgart			</td>
			<td>
				
					Magnus ? 200 kW pilot plant for SEG (Sorption Enhanced Reforming)				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4022/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					University of Idaho			</td>
			<td>
				
									
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3434/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_blue"></td>
						<td>
					University of Science and Technology of China			</td>
			<td>
				
					Hefei Facility				
			</td>
			<td>
					China                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3435/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Updraft gasifier, Waiariki			</td>
			<td>
				
					Windsor Engineering Group				
			</td>
			<td>
					New Zealand                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3104/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					UPM Biofuels			</td>
			<td>
				
					UPM Lappeenranta biorefinery				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3231/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					UPM Biofuels			</td>
			<td>
				
					UPM Rotterdam				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3880/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					UPM-Kymmene Oyj			</td>
			<td>
				
					Jamsankoski power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3616/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					UPM-Kymmene Oyj			</td>
			<td>
				
					Kaipola mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3608/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					UPM-Kymmene Oyj			</td>
			<td>
				
					Rauma mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3663/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					UPM-Kymmene Oyj			</td>
			<td>
				
					Kaukaa mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3645/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					UPM-Kymmene Oyj			</td>
			<td>
				
					Tervasaari mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3676/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					UPM-Kymmene Wood Oy			</td>
			<td>
				
					Joensuu plywood mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3618/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					UPM Rauma Paper mill			</td>
			<td>
				
					Rauma Paper Mill				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3661/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Uppsala Energi AB			</td>
			<td>
				
					Uppsula Energi				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3517/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					urbas Energietechnik			</td>
			<td>
				
					CHP Demonstrationsanlagen URBAS				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3098/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					USDA-ARS-ERRC			</td>
			<td>
				
									
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3436/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Valmet			</td>
			<td>
				
									
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3437/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vapo Oy			</td>
			<td>
				
					Forssan power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3638/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vapo Oy			</td>
			<td>
				
					Haapavesi power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3636/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vapo Oy			</td>
			<td>
				
					Kevatniemi power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3647/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vaskiluodon Voima Oy			</td>
			<td>
				
					Vaskiluoto 2 -power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3614/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vaskiluodon Voima Oy			</td>
			<td>
				
					Seinajoki power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3670/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vaskiluodon Voima Oy			</td>
			<td>
				
					Vaskiluoto 2 -power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3674/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vaskiluodon Voima Oy, Vaasa			</td>
			<td>
				
					Vaskiluodon Voima Biomass Gasification Plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3165/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vatajankosken S�hk� Oy			</td>
			<td>
				
					Kankaanpaa power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3623/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vattenfall			</td>
			<td>
				
					Amager #1				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3596/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vattenfall			</td>
			<td>
				
					Fynsvaerket #8				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3772/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Vattenfall			</td>
			<td>
				
					HySkies				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/4031/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vattenfall Kaukol�mp� Oy			</td>
			<td>
				
					Hameenlinna power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3635/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					VEAG			</td>
			<td>
				
					L�bbenau				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3486/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					VEAG			</td>
			<td>
				
					J�nschwalde				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3483/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					VEAG AG			</td>
			<td>
				
					n.a.				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3475/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Velocys			</td>
			<td>
				
					Altalto				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3866/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					Velocys			</td>
			<td>
				
					Bayou Fuels				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3867/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Verbio			</td>
			<td>
				
					VERBIO Nevada Biorefinery				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3350/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Verbund Austrian Hydropower AG			</td>
			<td>
				
					St. Andr�				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3465/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Verbund Austrian Hydropower AG			</td>
			<td>
				
					Biococomb				
			</td>
			<td>
					Austria                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3466/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Versalis / Eni			</td>
			<td>
				
					Crescentino Biorefinery				
			</td>
			<td>
					Italy                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3712/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Versa Renewables LLC			</td>
			<td>
				
					Versa Pyrolysis				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3406/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Verve Energy			</td>
			<td>
				
					Muja				
			</td>
			<td>
					Australia                                           			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3454/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Vieskan Voima Oy			</td>
			<td>
				
					Tulolantie heating plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3679/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Voimavasu Oy			</td>
			<td>
				
					Kantvik power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3627/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Voimavasu Oy			</td>
			<td>
				
					Salo power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3612/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Voimavasu Oy			</td>
			<td>
				
					Sakyla power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3613/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Voimavasu Oy			</td>
			<td>
				
					Salo power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3668/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Voimavasu Oy			</td>
			<td>
				
					Sakyla power plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3667/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Volter			</td>
			<td>
				
					 Kempele Ecovillage				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3169/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_orange"></td>
						<td>
					VTT Ltd.			</td>
			<td>
				
					VTT Ltd.				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3439/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					VTT Technical Research Centre of Finland Ltd			</td>
			<td>
				
					Dual fluidized-bed steam gasification pilot plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3162/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					VTT Technical Research Centre of Finland Ltd			</td>
			<td>
				
					Pressurized FB for synthesis gas production				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3163/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					VTT Technical Research Centre of Finland Ltd			</td>
			<td>
				
					Dual fluidized-bed steam gasification pilot plant				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3162/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					VTT Technical Research Centre of Finland Ltd			</td>
			<td>
				
					Pressurized FB for synthesis gas production				
			</td>
			<td>
					Finland                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3163/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					VVBGC AB			</td>
			<td>
				
					Vaexjoe Vaernamo Biomass Gasification Center AB				
			</td>
			<td>
					Sweden                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3120/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Waermeversorgung Grossenhain /POW AG			</td>
			<td>
				
					CHP Grossenhain				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3068/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Wegscheid Aschaffenburg			</td>
			<td>
				
					Wegscheid Aschaffenburg				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3749/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Wegscheid Bamberg			</td>
			<td>
				
					Wegscheid Bamberg				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3762/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Wegscheid Bayreuth			</td>
			<td>
				
					Wegscheid Bayreuth				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3739/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Wegscheid Demo			</td>
			<td>
				
					Wegscheid Demo				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3763/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Wegscheid Passau			</td>
			<td>
				
					Wegscheid Passau				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3755/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Weiss			</td>
			<td>
				
					Hillerod two stage gasifier				
			</td>
			<td>
					Denmark                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3113/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Welsh Power Group Ltd			</td>
			<td>
				
					Uskmouth				
			</td>
			<td>
					United Kingdom                                      			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3540/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					West Biofuels			</td>
			<td>
				
					LLC Thermal Reformer Synthesis West BiofuelsWoodland , CA				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3096/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Woodland Biofuels			</td>
			<td>
				
					pilot				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3346/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Woodland Biofuels			</td>
			<td>
				
					demo				
			</td>
			<td>
					Canada                                              			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3347/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					Woodpower in Wila			</td>
			<td>
				
					CHP Wila				
			</td>
			<td>
					Switzerland                                         			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3063/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_yellow"></td>
						<td>
					World Energy			</td>
			<td>
				
					Paramount Expansion				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3916/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					World Energy			</td>
			<td>
				
					Paramount				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3917/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					WUN Bioenergy			</td>
			<td>
				
					WUN Bioenergy				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3764/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Xylowatt, University Catholic of Louvain-la-Neuve (UCL)			</td>
			<td>
				
					Test Gasifier Plant TGP				
			</td>
			<td>
					Belgium                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3883/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					ZAB Balingen			</td>
			<td>
				
					KSV Balingen				
			</td>
			<td>
					Germany                                             			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3738/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					ZeaChem Inc.			</td>
			<td>
				
					Commercial scale biorefinery				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3266/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_red"></td>
						<td>
					ZEN-NOH			</td>
			<td>
				
					ZEN-NOH				
			</td>
			<td>
					Japan                                               			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3949/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
					<tr>
			<td class="status_green"></td>
						<td>
					Zilkha Biomass Energy			</td>
			<td>
				
					Zilkha Biomass-Crockett				
			</td>
			<td>
					United States                                       			</td>
						
			<td class="actions" style="width: 30px">
				<a href="/projects/info/3806/8JBaZy" class="btn btn-default btn-sm" role="button" data-toggle="modal" data-target="#viewproject">Info</a>			</td>
		</tr>
		
			</tbody>
	</table>
"""

## get data

In [59]:
from urllib.parse import urljoin
soup = BeautifulSoup(html_doc, 'html.parser')

# Find all <td> tags with class="actions"
action_tds = soup.find_all('td', class_='actions')

# Extract links from the <a> tags within the <td> tags
base_url = 'https://demoplants21.best-research.eu'
links = [urljoin(base_url, td.find('a')['href']) for td in action_tds]

print(links)

['https://demoplants21.best-research.eu/projects/info/3641/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3726/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3717/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3886/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3717/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3774/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3844/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3061/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3701/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3970/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3969/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3065/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3904/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3405/8JBaZy', 'https://demoplants21.best-research.eu/projects/info/3256/8JB

In [60]:
len(links)

750

## Extract info from Links

In [63]:
import requests
from bs4 import BeautifulSoup
import json

def extract_table_data(url,count):
    # Send a GET request to the URL
    print(f"urls {count} : {url}" )
    response = requests.get(url)
    base_url = "https://demoplants21.best-research.eu"
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the div with class "modal-body" and id "viewproject_content"
    div_content = soup.find('div', class_='modal-body', id='viewproject_content')

    # Create a dictionary to store the table data
    data = {}

    # Find all legends within the div
    legends = div_content.find_all('legend')

    # Loop through each legend and find its associated table
    for legend in legends:
        # Extract the text of the legend as the header
        header = legend.text.strip()

        # Find the next table sibling of the legend
        table = legend.find_next_sibling('div').find('table')

        # Extract table rows
        rows = {}
        if header not in ['Contact Information', 'Files']:
            for row in table.find_all('tr'):
                cells = row.find_all('td')
                # Use the first field as the key and the second field as the value
                if len(cells) == 2:
                    key = cells[0].text.strip()
                    value = cells[1].text.strip()
                    rows[key] = value
        elif header == 'Contact Information':
            # Treat contact information as a single key-value pair
            contact_info = table.find('td').text.strip()
            data[header] = contact_info
        elif header == 'Files':
            # Extract the filename from the "Files" section
            try:
                file_name = table.find('img')['src']
                data[header] = f'{base_url}{file_name}'
            except TypeError:
                data[header] = "No file available"

        # Add the header and table data to the dictionary
        if header not in ['Contact Information', 'Files']:
            data[header] = rows

    return data





In [64]:
all_data = {}
count=1
for link in links:
    url_data = extract_table_data(link,count)
    all_data[link] = url_data
    count += 1

# Save all data to a file
with open("all_data.json", "w") as file:
    json.dump(all_data, file, indent=4)

print("All data saved to all_data.json")


urls :  https://demoplants21.best-research.eu/projects/info/3641/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3726/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3717/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3886/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3717/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3774/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3844/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3061/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3701/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3970/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3969/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3065/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3904/8JBaZy
urls :  https://demoplants21.best-research.eu/projects/info/3405

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def extract_table_data(url):
    # Send a GET request to the URL
    response = requests.get(url)
    base_url = "https://demoplants21.best-research.eu"
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the div with class "modal-body" and id "viewproject_content"
    div_content = soup.find('div', class_='modal-body', id='viewproject_content')

    # Create a dictionary to store the table data
    data = {}

    # Find all legends within the div
    legends = div_content.find_all('legend')

    # Loop through each legend and find its associated table
    for legend in legends:
        # Extract the text of the legend as the header
        header = legend.text.strip()

        # Find the next table sibling of the legend
        table = legend.find_next_sibling('div').find('table')

        # Extract table rows
        rows = {}
        if header not in ['Contact Information', 'Files']:
            for row in table.find_all('tr'):
                cells = row.find_all('td')
                # Use the first field as the key and the second field as the value
                if len(cells) == 2:
                    key = cells[0].text.strip()
                    value = cells[1].text.strip()
                    rows[key] = value
        elif header == 'Contact Information':
            # Treat contact information as a single key-value pair
            contact_info = table.find('td').text.strip()
            data[header] = contact_info
        elif header == 'Files':
            # Extract the filename from the "Files" section
            file_name = table.find('img')['src']
            print(file_name)
            data[header] = f'{base_url}{file_name}'

        # Add the header and table data to the dictionary
        if header not in ['Contact Information', 'Files']:
            data[header] = rows

    return data

# Test the function
links=["https://demoplants21.best-research.eu/projects/info/3701/8JBaZy","https://demoplants21.best-research.eu/projects/info/3641/8JBaZy"]

all_data = {}

for link in links:
    url_data = extract_table_data(link)
    all_data[link] = url_data

# Save all data to a file
with open("all_data.json", "w") as file:
    json.dump(all_data, file, indent=4)

print("All data saved to all_data.json")


In [39]:
import requests
from bs4 import BeautifulSoup
import json

def extract_table_data(url):
    # Send a GET request to the URL
    response = requests.get(url)
    base_url = "https://demoplants21.best-research.eu"
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the div with class "modal-body" and id "viewproject_content"
    div_content = soup.find('div', class_='modal-body', id='viewproject_content')

    # Create a dictionary to store the table data
    data = {}

    # Find all legends within the div
    legends = div_content.find_all('legend')

    # Loop through each legend and find its associated table
    for legend in legends:
        # Extract the text of the legend as the header
        header = legend.text.strip()

        # Find the next table sibling of the legend
        table = legend.find_next_sibling('div').find('table')

        # Extract table rows
        rows = {}
        if header not in ['Contact Information', 'Files']:
            for row in table.find_all('tr'):
                cells = row.find_all('td')
                # Use the first field as the key and the second field as the value
                if len(cells) == 2:
                    key = cells[0].text.strip()
                    value = cells[1].text.strip()
                    rows[key] = value
        elif header == 'Contact Information':
            # Treat contact information as a single key-value pair
            contact_info = table.find('td').text.strip()
            data[header] = contact_info
        elif header == 'Files':
            # Extract the filename from the "Files" section
            file_name = table.find('img')['src']
            print(file_name)
            data[header] = f'{base_url}{file_name}'

        # Add the header and table data to the dictionary
        if header not in ['Contact Information', 'Files']:
            data[header] = rows

    # Convert the dictionary to JSON format
    json_data = json.dumps(data, indent=4)

    return json_data

# Test the function
links=["https://demoplants21.best-research.eu/projects/info/3701/8JBaZy","https://demoplants21.best-research.eu/projects/info/3641/8JBaZy"]

for link in links:
    json_result = extract_table_data(link)
    print(json_result)


/uploads/picture/3701/3701_2016-02-15-210702_picture.jpg
{
    "Geodata": {
        "Country": "Switzerland",
        "City": "Rheinfelden",
        "ZIP": "4310"
    },
    "Production": {
        "Type": "TRL 9 Commercial",
        "Technology": "PVC2: Power and Heat via Gasification",
        "Raw Material": "lignocellulosics",
        "Input 1": "Wood pellets (110 kg/h)",
        "Output 1": "power (electricity) (0.165 MWel )",
        "Output 2": "heat (0.26 MWth)",
        "Output additional information": "CHP Unit 0,165 Mwel + 0,26 MWth for district heating"
    },
    "Additional Information": {
        "Technology Brief": "Burkhardt pellet Gasifier CHP \r\n(fluidized bed in co current flow)",
        "Additional Information": "www.aew.ch/home.html\r\nhttp://burkhardt-energy.com/hp538/Technik.htm\nhttp://burkhardt-energy.com/hp538/Technik.htm"
    },
    "Contact Information": "marcel.kraenzlin@aew.ch \r\nlouis.luz@aew.ch",
    "Files": "https://demoplants21.best-research.eu/up

/uploads/picture/3701/3701_2016-02-15-210702_picture.jpg
{
    "Geodata": {
        "Country": "Switzerland",
        "City": "Rheinfelden",
        "ZIP": "4310"
    },
    "Production": {
        "Type": "TRL 9 Commercial",
        "Technology": "PVC2: Power and Heat via Gasification",
        "Raw Material": "lignocellulosics",
        "Input 1": "Wood pellets (110 kg/h)",
        "Output 1": "power (electricity) (0.165 MWel )",
        "Output 2": "heat (0.26 MWth)",
        "Output additional information": "CHP Unit 0,165 Mwel + 0,26 MWth for district heating"
    },
    "Additional Information": {
        "Technology Brief": "Burkhardt pellet Gasifier CHP \r\n(fluidized bed in co current flow)",
        "Additional Information": "www.aew.ch/home.html\r\nhttp://burkhardt-energy.com/hp538/Technik.htm\nhttp://burkhardt-energy.com/hp538/Technik.htm"
    },
    "Contact Information": "marcel.kraenzlin@aew.ch \r\nlouis.luz@aew.ch",
    "Files": "https://demoplants21.best-research.eu/up

## TO TABLE

In [37]:
from tabulate import tabulate
import json
import pandas as pd

# Your JSON data

# Convert JSON data to Python dictionary
data = json.loads(json_result)
table_data = []
for key, value in data.items():
    if isinstance(value, dict):
        for sub_key, sub_value in value.items():
            table_data.append([key + ' - ' + sub_key, sub_value])
    else:
        table_data.append([key, value])

# Convert list of lists to DataFrame
df = pd.DataFrame(table_data, columns=['Key', 'Value'])

# Display DataFrame using tabulate
print(tabulate(df, headers='keys', tablefmt='grid'))

+----+-------------------------------------------------+-----------------------------------------------------------------------------------------------+
|    | Key                                             | Value                                                                                         |
+====+=================================================+===============================================================================================+
|  0 | Geodata - Country                               | Switzerland                                                                                   |
+----+-------------------------------------------------+-----------------------------------------------------------------------------------------------+
|  1 | Geodata - City                                  | Rheinfelden                                                                                   |
+----+-------------------------------------------------+--------------------------

In [38]:
df

,Key,Value
0,Geodata - Country,Switzerland
1,Geodata - City,Rheinfelden
2,Geodata - ZIP,4310
3,Production - Type,TRL 9 Commercial
4,Production - Technology,PVC2: Power and Heat via Gasification
5,Production - Raw Material,lignocellulosics
6,Production - Input 1,Wood pellets (110 kg/h)
7,Production - Output 1,power (electricity) (0.165 MWel )
8,Production - Output 2,heat (0.26 MWth)
9,Production - Output additional information,"CHP Unit 0,165 Mwel + 0,26 MWth for district h..."


In [36]:
tabulate(table_data, headers=['Key', 'Value'], tablefmt='grid')

'+-------------------------------------------------+-----------------------------------------------------------------------------------------------+\n| Key                                             | Value                                                                                         |\n+=================================================+===============================================================================================+\n| Geodata - Country                               | Switzerland                                                                                   |\n+-------------------------------------------------+-----------------------------------------------------------------------------------------------+\n| Geodata - City                                  | Rheinfelden                                                                                   |\n+-------------------------------------------------+------------------------------------------------------